In [2]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.2MB 7.7MB/s 
     |████████████████████████████████| 491kB 44.4MB/s 
     |████████████████████████████████| 13.4MB 184kB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 
     |████████████████████████████████| 901kB 34.2MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=a5935ca79248d6bf82262a982265155462a97cf6efdc0decf28f7e6fc85774f4
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built ipadic


In [3]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# 文章の穴埋め

In [4]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
tokens

['今日', 'は', '[MASK]', 'へ', '行く', '。']

logitsで、語彙に含まれる各トークンの分類スコアを表すテンソルscoresが得られる

In [12]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

#BERTに入力して、分類スコアを得る
with torch.no_grad():
  output = bert_mlm(input_ids=input_ids)
  scores = output.logits

In [26]:
mask_position = input_ids[0].tolist().index(4)

# スコアが最も良いトークンのIDを取り出し、トークンに変換する
id_best = scores[0, mask_position].argmax(-1).item()
token_best  = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

text = text.replace('[MASK]', token_best)
text

'今日は東京へ行く。'

In [32]:
from mask_predict import predict_mask_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
text_topk

['今日は東京へ行く。',
 '今日はハワイへ行く。',
 '今日は学校へ行く。',
 '今日はニューヨークへ行く。',
 '今日はどこへ行く。',
 '今日は空港へ行く。',
 '今日はアメリカへ行く。',
 '今日は病院へ行く。',
 '今日はそこへ行く。',
 '今日はロンドンへ行く。']

In [38]:
from greedy import greedy_prediction

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [39]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は社会社会的な地位'

In [40]:
## ビムサーチ

In [47]:
def beam_search(text, tokenizer, bert_mlm, num_topk):
  num_mask = text.count('[MASK]')
  text_topk = [text]
  scores_topk = np.array([0])
  for _ in range(num_mask):
    text_candidates = []
    score_candidates = []
    for text_mask, score in zip(text_topk, scores_topk):
      text_topk_inner, scores_topk_inner = predict_mask_topk(
          text_mask, tokenizer, bert_mlm, num_topk
      )
      text_candidates.extend(text_topk_inner)
      score_candidates.append(score+scores_topk_inner)

    score_candidates = np.hstack(score_candidates)
    idx_list = score_candidates.argsort()[::-1][:num_topk]
    text_topk = [ text_candidates[idx] for idx in idx_list ]
    scores_topk = score_candidates[idx_list]

  return text_topk, scores_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk, scores_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')
print(*scores_topk)

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。
19.240992 18.098482 17.99474 17.338789 17.214308 17.02154 16.804558 16.644634 16.639458 16.619806


In [49]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk, scores_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。


In [13]:
import numpy as np

a = np.array([[3],[6464],[43],[46]])
a.argmax().item()

1